In [1]:
import tensorflow as tf
import keras
import os

## Create label dictionary
Since the price buckets may change, we'll create a dictionary to map the price buckets to a prettier format

In [8]:
# Get the buckets based on sub-folders of `buckets`
buckets = [f for f in os.listdir('buckets') if os.path.isdir(os.path.join('buckets', f))]

# Create a dictionary of classes where the key is the class name and the value is a formatted string of the class name
classes = {bucket: '$' + bucket.replace('_', '-$').title() for bucket in buckets}

{'25_30': '$25-$30',
 '35_40': '$35-$40',
 '10_15': '$10-$15',
 '20_25': '$20-$25',
 '45_50': '$45-$50',
 '30_35': '$30-$35',
 '15_20': '$15-$20',
 '40_45': '$40-$45'}

## Getting the data
We'll start by loading the data from the `buckets` folder

In [9]:
image_dim = 256

In [10]:
price_data = keras.utils.image_dataset_from_directory("buckets", labels="inferred", image_size=(image_dim,image_dim), batch_size=None)
price_data.cardinality()

Found 0 files belonging to 8 classes.


ValueError: No images found in directory buckets. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [10]:
price_data.shuffle(price_data.cardinality())
test_prices = price_data.take(1200) 
train_prices = price_data.skip(1200)
val_prices = train_prices.skip(600)
test_prices = test_prices.take(600)

test_prices.cardinality(), val_prices.cardinality(), train_prices.cardinality()

(<tf.Tensor: shape=(), dtype=int64, numpy=600>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2200>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2800>)

In [4]:
#reshape images (not necessary with this dataset, but you might enjoy this code for the future)
size = (image_dim, image_dim)
train_ds = train_prices.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = val_prices.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_prices.map(lambda x, y: (tf.image.resize(x, size), y))

In [5]:
## We'll also setup prefetching so that we can more easily load the data for training/testing
batch_size = 16
train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

### Building the model
In the first line we'll load the Xception model trained on Imagenet data. However we'll explicity drop the fully connected layers so we can add our own (seen in the `include_top=False` line)

In [6]:
# From Keras we load the Xception model
base_model = keras.applications.Xception(
    weights="imagenet",  # load the model with weights from training on ImageNet
    input_shape=(80, 80, 3), 
    include_top=False, # This states we won't include the "top" layers (the classifier)
)

# We can freeze the weights of any model/layer so they won't be updated when training the model
base_model.trainable = False

# With the base model defined we can create a regular Keras sequential model
inputs = keras.Input(shape=(80, 80, 3))
# This version of the Xception model expects inputs to be scaled from (0, 255) to a range of (-1., +1.)
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(inputs)

# The base model contains batchnorm layers, by setting training=False here it'll prevent those from being modified
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()
train_ds, validation_ds

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 80, 80, 3)]       0         
                                                                 
 rescaling (Rescaling)       (None, 80, 80, 3)         0         
                                                                 
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                             

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 80, 80, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 80, 80, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>)

In [27]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 1
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

ValueError: in user code:

    File "/Users/ozzysimpson/anaconda3/envs/Neural/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/ozzysimpson/anaconda3/envs/Neural/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/ozzysimpson/anaconda3/envs/Neural/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/ozzysimpson/anaconda3/envs/Neural/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Users/ozzysimpson/anaconda3/envs/Neural/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/o

In [26]:
import matplotlib.pyplot as plt
import numpy as np

print(type(test_ds))
print(type(test_ds.take(1)))
print(test_ds)

plt.figure(figsize=(10, 10))
for ix, (images, labels) in enumerate(train_ds.take(1)):
    model_preds = model.predict(images)
    model_labels = np.where(model_preds>0, 1, 0)
    for iy in range(9):
        ax = plt.subplot(3, 3, iy + 1)
        plt.imshow(np.array(images[iy]).astype(int))
        plt.title(f"Actual: {int(labels[iy])} - Predicted: {model_labels[iy]}")
        plt.axis("off")

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>
<class 'tensorflow.python.data.ops.take_op._TakeDataset'>
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 80, 80, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


ValueError: in user code:

    File "/Users/ozzysimpson/anaconda3/envs/Neural/lib/python3.10/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/Users/ozzysimpson/anaconda3/envs/Neural/lib/python3.10/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/ozzysimpson/anaconda3/envs/Neural/lib/python3.10/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/ozzysimpson/anaconda3/envs/Neural/lib/python3.10/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/Users/ozzysimpson/anaconda3/envs/Neural/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Us

<Figure size 1000x1000 with 0 Axes>